In [1]:
import geopandas
from matplotlib import pyplot as plt
import pandas as pd

import numpy as np
from shapely.ops import cascaded_union
from shapely import wkt
#pip install folium matplotlib mapclassify

# Loading the Catalan Map

In [2]:
# We upload the Catalan map with all subregions (called Comarques)
gdf_comarcas = geopandas.read_file('Comarcas_CAT.json')
fig = plt.figure()
fig.set_figheight(10)
fig.set_figwidth(15)

<Figure size 1080x720 with 0 Axes>

In [3]:
gdf_comarcas.head()

Comarca_Index    Comarca_Name  \
0              1        Alt Camp   
1              2     Alt Empordà   
2              3     Alt Penedès   
3              4      Alt Urgell   
4              5  Alta Ribagorça   

                                            geometry  
0  MULTIPOLYGON (((1.48699 41.46955, 1.48703 41.4...  
1  MULTIPOLYGON (((3.22338 42.23682, 3.22336 42.2...  
2  MULTIPOLYGON (((1.61690 41.51260, 1.61712 41.5...  
3  MULTIPOLYGON (((1.31941 41.99157, 1.32141 41.9...  
4  MULTIPOLYGON (((0.87950 42.63140, 0.87967 42.6...

# Defining what subregions compose each supraregion

In [4]:
#Now I load the Touristic Brand dataset that defines which group of subregions compose each supraregion.
mturistiques = pd.read_csv('Mturistiques.csv',delimiter=';')
df = gdf_comarcas.merge(mturistiques, left_on='Comarca_Name', right_on = 'Comarca',  how='left')
gdf = geopandas.GeoDataFrame(df, crs='epsg:4326')
gdf.head()

Comarca_Index    Comarca_Name  \
0              1        Alt Camp   
1              2     Alt Empordà   
2              3     Alt Penedès   
3              4      Alt Urgell   
4              5  Alta Ribagorça   

                                            geometry         Comarca  \
0  MULTIPOLYGON (((1.48699 41.46955, 1.48703 41.4...        Alt Camp   
1  MULTIPOLYGON (((3.22338 42.23682, 3.22336 42.2...     Alt Empordà   
2  MULTIPOLYGON (((1.61690 41.51260, 1.61712 41.5...     Alt Penedès   
3  MULTIPOLYGON (((1.31941 41.99157, 1.32141 41.9...      Alt Urgell   
4  MULTIPOLYGON (((0.87950 42.63140, 0.87967 42.6...  Alta Ribagorça   

   Marca Turística  
0    Costa Daurada  
1      Costa Brava  
2  Costa Barcelona  
3         Pirineus  
4         Pirineus

## We observe all touristic brand and their respective subregions

In [5]:
#gdf.explore('Marca Turística')

# One Supraregion is obtained 

In [6]:
#Now we try to get one of the touristic brands: Costa Brava
gdf = gdf[['Comarca_Index','Comarca_Name','Marca Turística','geometry']]
gdf_costa_brava = gdf[gdf['Marca Turística']=='Costa Brava']
gdf_costa_brava

Comarca_Index     Comarca_Name Marca Turística  \
1               2      Alt Empordà     Costa Brava   
9              10     Baix Empordà     Costa Brava   
19             20          Gironès     Costa Brava   
27             28  Pla de l'Estany     Costa Brava   
33             34            Selva     Costa Brava   

                                             geometry  
1   MULTIPOLYGON (((3.22338 42.23682, 3.22336 42.2...  
9   MULTIPOLYGON (((3.21899 42.04774, 3.21898 42.0...  
19  MULTIPOLYGON (((2.92889 42.11487, 2.92898 42.1...  
27  MULTIPOLYGON (((2.78415 42.20619, 2.78427 42.2...  
33  MULTIPOLYGON (((2.91480 41.70902, 2.91479 41.7...

In [7]:
# We define a function so not to repeat the process for each supraregion.
def boundary_generator(df,string):
    polygons = gdf.loc[(gdf['Marca Turística'] == string)]
    polygons = polygons['geometry'].unary_union
    
    return polygons

In [8]:
# Costa Brava
boundary_costabrava = boundary_generator(df,'Costa Brava')



In [9]:
data = {'Marca Turistica': [
                            'Costa Brava'
                           ],
        'geometry': [
                     boundary_costabrava
                    ],
       'code':[
               1
              ],
       }

# Create DataFrame
df_marques = pd.DataFrame(data)

gp_marques = geopandas.GeoDataFrame(df_marques, crs='epsg:4326')
#gp_marques.explore()

/Users/josepferrersanchez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


# The process is repeated for each supraregion

In [10]:

# Costa Brava
boundary_costabrava = boundary_generator(df,'Costa Brava')

# Costa Daurada
boundary_costadaurada = boundary_generator(df,'Costa Daurada')

# Barcelona
boundary_barcelona = boundary_generator(df,'Barcelona')

# Terres de l'Ebre
boundary_ebre = boundary_generator(df,"Terres de l'Ebre")

# Terres de Lleida
boundary_lleida = boundary_generator(df,"Terres de Lleida")

# Pirineus
boundary_pirineus = boundary_generator(df,"Pirineus")

# Paisatges de Barcelona
boundary_paisatges_barcelona= boundary_generator(df,"Paisatges Barcelona")

# Costa de Barcelona
boundary_costa_barcelona= boundary_generator(df,"Costa Barcelona")

# Aran
boundary_vall_aran= boundary_generator(df,"Val d'Aran")


In [11]:
# initialize data of lists.
data = {'Marca Turistica': [
                            'Costa Brava', 'Costa Daurada', 'Barcelona', "Terres de l'Ebre",
                            "Terres de Lleida", "Pirineus", "Paisatges de Barcelona",
                            "Costa Barcelona", "Vall d'Aran"
                           ],
        'geometry': [
                     boundary_costabrava, boundary_costadaurada, boundary_barcelona,
                     boundary_ebre, boundary_lleida, boundary_pirineus,
                     boundary_paisatges_barcelona, boundary_costa_barcelona,
                     boundary_vall_aran
                    ],
       'code':[
               1,2,3,4,5,6,7,8,9
              ],
       }
 
# Create DataFrame
df_marques = pd.DataFrame(data)

gp_marques = geopandas.GeoDataFrame(df_marques, crs='epsg:4326')
#gp_marques.explore()

/Users/josepferrersanchez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


In [12]:
#We obtain the dataset with all 9 supraregions and their corresponding polygon. 
gp_marques[['Marca Turistica','code','geometry']]

Marca Turistica  code  \
0             Costa Brava     1   
1           Costa Daurada     2   
2               Barcelona     3   
3        Terres de l'Ebre     4   
4        Terres de Lleida     5   
5                Pirineus     6   
6  Paisatges de Barcelona     7   
7         Costa Barcelona     8   
8             Vall d'Aran     9   

                                            geometry  
0  MULTIPOLYGON (((2.79130 41.67010, 2.79130 41.6...  
1  MULTIPOLYGON (((0.87187 40.95091, 0.87186 40.9...  
2  MULTIPOLYGON (((2.15247 41.32297, 2.15250 41.3...  
3  MULTIPOLYGON (((0.55225 40.57260, 0.55225 40.5...  
4  POLYGON ((1.03838 41.49671, 1.03847 41.49660, ...  
5  MULTIPOLYGON (((1.64866 41.95634, 1.64853 41.9...  
6  POLYGON ((1.42973 41.80196, 1.42996 41.80193, ...  
7  MULTIPOLYGON (((1.66160 41.19511, 1.66159 41.1...  
8  POLYGON ((0.71604 42.85832, 0.71633 42.85810, ...